In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
import folium 
import requests

In [52]:
df_aviones = pd.read_pickle('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aviones_limpio.pkl')


In [53]:
print(df_aviones.columns)

Index(['codigo_aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
       'hora_salida_programada', 'hora_salida_real',
       'duracion_programada_vuelo', 'duracion_real', 'retraso_salida',
       'hora_despegue', 'tiempo_pista_salida', 'tiempo_retraso_aerolinea',
       'tiempo_retraso_clima', 'tiempo_retraso_sistema_aviacion',
       'tiempo_retraso_seguridad', 'retraso_llegada_aeronave', 'ciudad_origen',
       'estado_origen', 'aeropuerto_origen', 'dia_semana', 'año',
       'fin_de_semana', 'aeropuerto_destino', 'distancia_millas'],
      dtype='object')


In [54]:
print(df_aviones.head())

  codigo_aerolinea      fecha  numero_vuelo numero_cola  \
0        Envoy Air 2021-12-01          3632      N202NN   
1        Envoy Air 2021-12-01          3647      N653AE   
2        Envoy Air 2021-12-01          3723      N616AE   
3        Envoy Air 2021-12-01          4192      N681AE   
4        Envoy Air 2021-12-01          4311      N687JS   

  hora_salida_programada hora_salida_real  duracion_programada_vuelo  \
0                  08:11            08:16                         57   
1                  06:45            06:37                         56   
2                  14:47            14:38                         54   
3                  12:35            12:35                         61   
4                  18:36            19:47                         62   

   duracion_real  retraso_salida hora_despegue  ...  tiempo_retraso_seguridad  \
0             69               5         08:33  ...                         0   
1             61              -8         06:50  ..

In [55]:
# Obteniendo los valores únicos de la columna 'codigo_aeropuerto'
valores_unicos_aeropuertos = df_aviones['aeropuerto_destino'].unique()

# Imprimiendo los valores únicos
print(valores_unicos_aeropuertos)

# Si quieres contar cuántos aeropuertos únicos tienes, puedes usar la función len()
print(f"Número de aeropuertos únicos: {len(valores_unicos_aeropuertos)}")


['Dallas/Fort Worth International' "Chicago O'Hare International"
 'Charlotte Douglas International' 'Ronald Reagan Washington National'
 'Philadelphia International' 'Hartsfield-Jackson Atlanta International'
 'William P Hobby' 'Baltimore/Washington International Thurgood Marshall'
 'Orlando International' 'Nashville International'
 'Cincinnati/Northern Kentucky International' 'Indianapolis International'
 'Pittsburgh International' 'Bishop International'
 'Washington Dulles International' 'Norfolk International'
 'Miami International' 'Austin - Bergstrom International'
 'Phoenix Sky Harbor International' 'Charlottesville Albemarle'
 'Shreveport Regional' 'Springfield-Branson National' 'Valdosta Regional'
 'Albert J Ellis' 'Baton Rouge Metropolitan/Ryan Field'
 'Central Il Regional Airport at Bloomington' 'Key West International'
 'Blue Grass' 'West Virginia International Yeager' 'Lovell Field'
 'Southwest Georgia Regional' 'Dothan Regional'
 'Augusta Regional at Bush Field' 'Lehigh V

In [56]:
# Paso 1: Crear DataFrames temporales para origen y destino
df_origen = df_aviones[['aeropuerto_origen', 'ciudad_origen', 'estado_origen']].copy()
df_destino = df_aviones[['aeropuerto_destino']].copy()

# Renombrar columnas de df_destino para que coincidan con las de df_origen
df_destino.columns = ['aeropuerto_origen']

# Añadir columnas vacías de ciudad y estado en df_destino para igualar estructuras
df_destino['ciudad_origen'] = None
df_destino['estado_origen'] = None

# Paso 2: Concatenar los DataFrames
df_aeropuertos_concatenados = pd.concat([df_origen, df_destino])

# Paso 3: Eliminar duplicados
df_aeropuertos_unicos = df_aeropuertos_concatenados.drop_duplicates(subset=['aeropuerto_origen'])

# Reseteando el índice del DataFrame final
df_aeropuertos_unicos.reset_index(drop=True, inplace=True)

# Mostrando el nuevo DataFrame
print(df_aeropuertos_unicos)


                            aeropuerto_origen      ciudad_origen estado_origen
0                            Abilene Regional            Abilene            TX
1                       Akron-Canton Regional              Akron            OH
2    Hartsfield-Jackson Atlanta International            Atlanta            GA
3    Jackson Medgar Wiley Evers International  Jackson/Vicksburg            MS
4                  Jacksonville International       Jacksonville            FL
..                                        ...                ...           ...
358                     Long Island MacArthur              Islip            NY
359             Ithaca Tompkins International    Ithaca/Cortland            NY
360                              Jackson Hole            Jackson            WY
361                         Aberdeen Regional           Aberdeen            SD
362                               Garden City               None          None

[363 rows x 3 columns]


In [57]:
# Localizar las filas donde el aeropuerto es "Garden City" y actualizar 'ciudad_origen' y 'estado_origen'
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City", 'ciudad_origen'] = "Garden City"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City", 'estado_origen'] = "KS"

# Verificar los cambios
print(df_aeropuertos_unicos[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City"])



    aeropuerto_origen ciudad_origen estado_origen
362       Garden City   Garden City            KS


In [58]:
# Cambiar los nombres de las columnas
df_aeropuertos_unicos.rename(columns={
    'aeropuerto_origen': 'nombre_aeropuerto',
    'ciudad_origen': 'ciudad',
    'estado_origen': 'estado'
}, inplace=True)

# Verificar los cambios mostrando los nuevos nombres de las columnas
print(df_aeropuertos_unicos.columns)


Index(['nombre_aeropuerto', 'ciudad', 'estado'], dtype='object')


C:\Users\34670\AppData\Local\Temp\ipykernel_1364\3509175004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos.rename(columns={


In [59]:
# Tus credenciales de la API de Foursquare
CLIENT_ID = "QCDWZLNWZBWTQKPLJLC2GCFIGRICGGQX1D1AJD1JUM0FMUPU"
CLIENT_SECRET = "NVRHLER4BNQ2LHGUEWDFPAIF132KBCT2JHQU5X35MLCS1S0B"
API_KEY = "fsq3gHQLqKS5DaSzw9RzKUqjuW9Ec14h06kJE5xVxKSHAfg="

# Headers para la solicitud
headers = {"Accept": "application/json", "Authorization": API_KEY}

# Preparar el DataFrame para almacenar los datos de las coordenadas
df_aeropuertos_unicos['latitude'] = None
df_aeropuertos_unicos['longitude'] = None
df_aeropuertos_unicos['direccion'] = None

# Iterar sobre cada fila del DataFrame
for index, row in df_aeropuertos_unicos.iterrows():
    # Parámetros para la búsqueda
    url_params = {
        "query": "airport" + row['nombre_aeropuerto'],
        "near": f"{row['ciudad']}, {row['estado']}",
        "limit": 1
    }

    # Hacer la llamada a la API
    response = requests.get(url="https://api.foursquare.com/v3/places/search", params=url_params, headers=headers)

    # Verificar que la respuesta es exitosa
    if response.status_code == 200:
        data = response.json()

        # Extraer los datos si hay resultados
        if data['results']:
            result = data['results'][0]  # Tomar el primer resultado
            latitude = result['geocodes']['main']['latitude']
            longitude = result['geocodes']['main']['longitude']
            direccion = result['location']['formatted_address']

            # Guardar los datos en el DataFrame
            df_aeropuertos_unicos.at[index, 'latitude'] = latitude
            df_aeropuertos_unicos.at[index, 'longitude'] = longitude
            df_aeropuertos_unicos.at[index, 'direccion'] = direccion
    else:
        print(f"Error en la fila {index} con el aeropuerto {row['nombre_aeropuerto']}. Respuesta: {response.status_code}")

# Verificar los datos actualizados
print(df_aeropuertos_unicos.head())


C:\Users\34670\AppData\Local\Temp\ipykernel_1364\3202471268.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos['latitude'] = None
C:\Users\34670\AppData\Local\Temp\ipykernel_1364\3202471268.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aeropuertos_unicos['longitude'] = None
C:\Users\34670\AppData\Local\Temp\ipykernel_1364\3202471268.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

                          nombre_aeropuerto             ciudad estado  \
0                          Abilene Regional            Abilene     TX   
1                     Akron-Canton Regional              Akron     OH   
2  Hartsfield-Jackson Atlanta International            Atlanta     GA   
3  Jackson Medgar Wiley Evers International  Jackson/Vicksburg     MS   
4                Jacksonville International       Jacksonville     FL   

    latitude  longitude                                          direccion  
0       None       None                                               None  
1  41.080564 -81.519677                     182 S Main St, Akron, OH 44308  
2  33.640846 -84.432594  6000 N Terminal Pkwy (at Airport Loop Rd), Atl...  
3       None       None                                               None  
4  30.328992 -81.657959  303 N Laura St (Duval St.), Jacksonville, FL 3...  


In [64]:
df_aeropuertos_unicos.to_excel('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.xlsx', index=False)

In [63]:
# Segunda iteración para las filas sin resultados
for index, row in df_aeropuertos_unicos.iterrows():
    # Verificar si la fila no tiene coordenadas o dirección
    if pd.isnull(row['latitude']) or pd.isnull(row['longitude']) or pd.isnull(row['direccion']):
        # Modificar la búsqueda añadiendo "airport" al inicio
        url_params['query'] = "airport " + row['nombre_aeropuerto']

        # Hacer la llamada a la API
        response = requests.get(url="https://api.foursquare.com/v3/places/search", params=url_params, headers=headers)

        # Verificar que la respuesta es exitosa
        if response.status_code == 200:
            data = response.json()

            # Extraer los datos si hay resultados
            if data['results']:
                result = data['results'][0]  # Tomar el primer resultado nuevamente
                latitude = result['geocodes']['main']['latitude']
                longitude = result['geocodes']['main']['longitude']
                direccion = result['location']['formatted_address']

                # Actualizar los datos en el DataFrame
                df_aeropuertos_unicos.at[index, 'latitude'] = latitude
                df_aeropuertos_unicos.at[index, 'longitude'] = longitude
                df_aeropuertos_unicos.at[index, 'direccion'] = direccion
        else:
            print(f"Error en la segunda búsqueda en la fila {index} con el aeropuerto {row['nombre_aeropuerto']}. Respuesta: {response.status_code}")

# Verificar los datos actualizados después de la segunda pasada
print(df_aeropuertos_unicos.head())


                          nombre_aeropuerto             ciudad estado  \
0                          Abilene Regional            Abilene     TX   
1                     Akron-Canton Regional              Akron     OH   
2  Hartsfield-Jackson Atlanta International            Atlanta     GA   
3  Jackson Medgar Wiley Evers International  Jackson/Vicksburg     MS   
4                Jacksonville International       Jacksonville     FL   

    latitude   longitude                                          direccion  
0  37.967543 -100.871894             205 E Laurel St, Garden City, KS 67846  
1  41.080564  -81.519677                     182 S Main St, Akron, OH 44308  
2  33.640846  -84.432594  6000 N Terminal Pkwy (at Airport Loop Rd), Atl...  
3  37.974664 -100.871909              1011 N Main St, Garden City, KS 67846  
4  30.328992  -81.657959  303 N Laura St (Duval St.), Jacksonville, FL 3...  


In [66]:
# Crear el mapa base
mapa = folium.Map(location=[40, -95], zoom_start=4)  # Puedes ajustar la ubicación y el zoom inicial según tus necesidades

# Añadir marcadores para cada aeropuerto
for index, row in df_aeropuertos_unicos.iterrows():
    folium.Marker(
        [row['latitude'], row['longitude']], 
        popup=f"{row['nombre_aeropuerto']}", 
        tooltip=row['nombre_aeropuerto']
    ).add_to(mapa)

# Mostrar el mapa
mapa

In [68]:
# Para guardar como CSV
df_aeropuertos_unicos.to_csv('C:/Users/34670/Desktop/python/Hack a boss/proyecto_3/streamlit/proyecto_aviones/data/aeropuertos_unicos.csv', index=False)
